In [ ]:
import numpy as np 
import pickle
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import numpy as np 
import pickle
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tensorflow.keras.preprocessing import image
import copy
from tensorflow.keras.applications.resnet50 import preprocess_input

In [ ]:
df = pd.read_csv("/kaggle/input/nih-chest-xrays-tfrecords/preprocessed_data.csv")

In [ ]:
images_data = "../input/nih-chest-xrays-tfrecords/data"

In [ ]:
feature_map = {
    'image': tf.io.FixedLenFeature([], tf.string),
    'image_id': tf.io.FixedLenFeature([], tf.string),
    'No Finding': tf.io.FixedLenFeature([], tf.int64),
    'Atelectasis': tf.io.FixedLenFeature([], tf.int64),
    'Consolidation': tf.io.FixedLenFeature([], tf.int64),
    'Infiltration': tf.io.FixedLenFeature([], tf.int64),
    'Pneumothorax': tf.io.FixedLenFeature([], tf.int64),
    'Edema': tf.io.FixedLenFeature([], tf.int64),
    'Emphysema': tf.io.FixedLenFeature([], tf.int64),
    'Fibrosis': tf.io.FixedLenFeature([], tf.int64),
    'Effusion': tf.io.FixedLenFeature([], tf.int64),
    'Pneumonia': tf.io.FixedLenFeature([], tf.int64),
    'Pleural_Thickening': tf.io.FixedLenFeature([], tf.int64),
    'Cardiomegaly': tf.io.FixedLenFeature([], tf.int64),
    'Nodule': tf.io.FixedLenFeature([], tf.int64),
    'Mass': tf.io.FixedLenFeature([], tf.int64),
    'Hernia': tf.io.FixedLenFeature([], tf.int64)
}

In [ ]:
def tfr_decoder(path, shuffle=True):
    def image_decoder(data):
        _ = tf.io.parse_single_example(data, feature_map) 
        image = _['image']
        image = tf.io.decode_image(image, channels=3)
        image = tf.image.convert_image_dtype(image, tf.float32)
        image = tf.image.resize_with_pad(image, 150, 150)
        image.set_shape([150,150,3])
        image = image/255.
        
        print([label for label in sorted(list(_.keys())) if label!='image' and label!='image_id'])
        labels = [tf.cast(_[x], tf.float32) for x in sorted(list(_.keys())) if x!='image_id' and x!='image']
        
        return image, labels
    
    data_list = [os.path.join(images_data,x) for x in os.listdir(path)]
    split = int(len(data_list)*0.8)
    train_data, val_data = data_list[:split], data_list[split:]
    
    trainds = tf.data.TFRecordDataset(train_data)
    trainds = trainds.map(image_decoder, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    
    valds = tf.data.TFRecordDataset(val_data)
    valds = valds.map(image_decoder, num_parallel_calls=tf.data.experimental.AUTOTUNE)

    if shuffle:
        trainds = trainds.shuffle(1024)
        
    trainds = trainds.batch(128).prefetch(tf.data.experimental.AUTOTUNE)
    valds = valds.batch(128).prefetch(tf.data.experimental.AUTOTUNE)
    return trainds, valds

In [ ]:
training , validation = tfr_decoder(images_data)

In [ ]:
effic = tf.keras.applications.EfficientNetB2(
    include_top=False, weights=None,input_shape=(150,150,3), pooling='avg')

incep = tf.keras.applications.InceptionV3(
    include_top=False, weights=None, input_shape=(150,150,3), pooling='avg')

In [ ]:
try:
    model = tf.keras.models.load_model('../input/temp12/MyModel')
except Exception as e:
    print(e)
    model = tf.keras.Sequential([
                incep,
                tf.keras.layers.Dense(512, activation='relu'),
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.Dense(128, activation='relu'),
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.Dense(32, activation='relu'),
                tf.keras.layers.Dense(15, activation='sigmoid'),
    ])

    model.compile(loss='binary_crossentropy',
                optimizer='Adam',
                metrics=[tf.keras.metrics.AUC(multi_label=True),'binary_accuracy'])
    model.fit(training, epochs=25)
    model.save("../input/temp12/MyModel")

In [ ]:
label_list = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion', 'Emphysema', 'Fibrosis', 
              'Hernia', 'Infiltration', 'Mass', 'No Finding', 'Nodule', 'Pleural_Thickening', 'Pneumonia', 
              'Pneumothorax']
columns = {}
for label in label_list:
    columns[label] = []
    
for element in validation.as_numpy_iterator():
    for label in element[1]:
        for n, data in enumerate(label):
            columns[label_list[n]].append(data)
    
valdf = pd.DataFrame(columns)
valdf

In [ ]:
Example = list(valdf.loc[valdf['Atelectasis'] == 1].head(10).index)

In [ ]:
val_image_data = []
for element in validation.as_numpy_iterator():
    for img in element[0]:
        img = img*255.0
        val_image_data.append(img)

In [ ]:
def interpolate_images(baseline,image,alphas):
    alphas_x = alphas[:, tf.newaxis, tf.newaxis, tf.newaxis]
    baseline_x = tf.expand_dims(baseline, axis=0)
    input_x = tf.expand_dims(image, axis=0)
    delta = input_x - baseline_x
    images = baseline_x +  alphas_x * delta
    return images

In [ ]:
def compute_gradients(images, target_class_idx):
    with tf.GradientTape() as tape:
        tape.watch(images)
        logits = model(images)
        probs = tf.nn.softmax(logits, axis=-1)[:,target_class_idx]
        return tape.gradient(probs, images)

In [ ]:
def integral_approximation(gradients):
    # riemann_trapezoidal
    grads = (gradients[:-1] + gradients[1:]) / tf.constant(2.0)
    integrated_gradients = tf.math.reduce_mean(grads, axis=0)
    return integrated_gradients

In [ ]:
@tf.function
def integrated_gradients(baseline,image,target_class_idx, m_steps=150, batch_size=64):
    # 1. Generate alphas
    alphas = tf.linspace(start=0.0, stop=1.0, num=m_steps)

    # Accumulate gradients across batches
    integrated_gradients = 0.0

    # Batch alpha images
    ds = tf.data.Dataset.from_tensor_slices(alphas).batch(batch_size)

    for batch in ds:

        # 2. Generate interpolated images
        batch_interpolated_inputs = interpolate_images(baseline=baseline,
                                                   image=image,
                                                   alphas=batch)

        # 3. Compute gradients between model outputs and interpolated inputs
        batch_gradients = compute_gradients(images=batch_interpolated_inputs, 
                                            target_class_idx=target_class_idx)

        # 4. Average integral approximation. Summing integrated gradients across batches.
        integrated_gradients += integral_approximation(gradients=batch_gradients)

    # 5. Scale integrated gradients with respect to input
    scaled_integrated_gradients = (image - baseline) * integrated_gradients
    return scaled_integrated_gradients

In [ ]:
def plot_img_attributions(baseline,image,target_class_idx, m_steps=tf.constant(50),cmap=None,
                          overlay_alpha=0.4):

    attributions = integrated_gradients(baseline=baseline,image=image,target_class_idx=target_class_idx,
                                      m_steps=m_steps)

    # Sum of the attributions across color channels for visualization.
    # The attribution mask shape is a grayscale image with height and width
    # equal to the original image.
    attribution_mask = tf.reduce_sum(tf.math.abs(attributions), axis=-1)

    fig, axs = plt.subplots(nrows=2, ncols=2, squeeze=False, figsize=(8, 8))

    axs[0, 0].set_title('Baseline image')
    axs[0, 0].imshow(baseline)
    axs[0, 0].axis('off')

    axs[0, 1].set_title('Original image')
    axs[0, 1].imshow(image)
    axs[0, 1].axis('off')

    axs[1, 0].set_title('Attribution mask')
    axs[1, 0].imshow(attribution_mask, cmap=cmap)
    axs[1, 0].axis('off')

    axs[1, 1].set_title('Overlay')
    axs[1, 1].imshow(attribution_mask, cmap=cmap)
    axs[1, 1].imshow(image, alpha=overlay_alpha)
    axs[1, 1].axis('off')

    plt.tight_layout()
    return fig

In [ ]:
from tensorflow.keras.preprocessing import image
import copy
from tensorflow.keras.applications.resnet50 import preprocess_input
model = tf.keras.models.load_model('../input/temp12/MyModel')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
baseline = tf.zeros(shape=(150,150,3))
for num in Example:
    _ = plot_img_attributions(image=val_image_data[num],baseline=baseline,target_class_idx=0, m_steps=2400,
                              cmap=plt.cm.inferno, overlay_alpha=0.4)

In [ ]:
from tensorflow.keras.preprocessing import image
import copy
from tensorflow.keras.applications.resnet50 import preprocess_input
def plot1(image , prediction ):
    _ = plot_img_attributions(image=image,baseline=baseline,target_class_idx=prediction, m_steps=2400,
                                  cmap=plt.cm.inferno, overlay_alpha=0.4)
    
def TestSingle(image):
    images_list = []
    images_list.append(np.array(image))
    x = np.asarray(images_list)
    model.predict(x)
    prediction = model.predict(x)
    plot1(image,np.argmax(prediction))
    label_list = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion', 'Emphysema', 'Fibrosis', 
                  'Hernia', 'Infiltration', 'Mass', 'No Finding', 'Nodule', 'Pleural_Thickening', 'Pneumonia', 
                  'Pneumothorax']
    print  (label_list[np.argmax(prediction)])
TestSingle(val_image_data[308])

In [ ]:
import visualkeras

visualkeras.layered_view(model)